<h1 align=center> Implementation of ZKP for Merkle tree Using Python</h1>

.

In [97]:
from hashlib import*
from math import*
from csv import *
from hashlib import * 
import os
import random

In [98]:
L=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,"Nguepi","Nguefack","Tondji"]
L1=[1,2,3,4]


In [99]:
# This code allow us to integer hash  
def hashThis_int(M):
    r=0
    hash=sha256();
    hash.update(str(r).encode());
    hash.update(M.encode());
    return int(hash.hexdigest(),16);

In [100]:
hashThis_int("Nguefack")

98308941888556364773191993917330179511595787205275262433735289253208501263596L

In [101]:
## Check if the number is the power of two or not.
def Check_power_two(x):
    n=log(x)/log(2)
    if n.is_integer():
        return True
    return False

In [102]:
#This function transform the content of csv file into a list.
def Transform_csv_file_on_list(filePath):
    filePath = '/home/nguepi/Bureau/Test.csv'
    
    fileOpen = open(filePath, 'rU')
    # opening the file for reading in Universal NewLine (rU) 

    fileReader = reader(fileOpen)
    # initializing the CSV Reader for traversing data inside the csv file

    data_list = []
    for row in fileReader :

        for tnx in row :

            currentItem = str(tnx)
            data_list.append(currentItem)
    return data_list

In [103]:
filePath = '/home/nguepi/Bureau/Test.csv'
Transform_csv_file_on_list(filePath)

['nguepi',
 'nguepi',
 'nguepi',
 'nguepi',
 'nguepi',
 'nguepi',
 'nguepi',
 'nguepi',
 'nguepi',
 'nguepi',
 'nguepi',
 'idriss',
 'idriss',
 'idriss',
 'idriss',
 'idriss',
 'idriss',
 'idriss',
 'idriss',
 'idriss',
 'idriss',
 'idriss',
 'idriss',
 'nguefack',
 'nguefack',
 'nguefack',
 'nguefack',
 'nguefack',
 'nguefack',
 'nguefack',
 'nguefack',
 'nguefack']

In [104]:
## This function allow us to solve padding probleme.
def Check_data_len(L):
    while Check_power_two(len(L))==False:
        L.append(random.randrange(0, 10000, 1))
    return L       

In [105]:
## This function allow us to hash all the elements of the list L 
def Return_Hashlist(L):
    List_result=[]
    for i in range(len(L)):
        List_result.append(sha256(str(L[i])).hexdigest())
    return List_result  

In [106]:
## This function allow us to add an element L in the dictionnary Dict at the position i
def Add_list(L,Dict,i):
    Dict[str(i)]=L
    return Dict

In [107]:
#Give the hash of the concatenation of H1 and H2.
def Add_and_hash(H1,H2):
    return sha256(str(hashThis_int(H1)+hashThis_int(H2))).hexdigest()

In [108]:
# This code allow us to build the Merkle tree
def Merkle_tree(L):
    j=2
    Dict={}
    Check_data_len(L)
    Dict[str(0)]=L
    L1=Return_Hashlist(L)
    Add_list(L1,Dict,1)
    while j<=int(log(len(L))/log(2)+1):
        Lis=[]
        for i in range(((len(L1)-1)//2)+1):
            Lis.append(Add_and_hash(L1[2*i],L1[2*i+1]))
        Add_list(Lis,Dict,j) 
        L1=Lis
        j+=1  
    return Dict

In [109]:
d=Merkle_tree(L)

In [110]:
d

{'0': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  'Nguepi',
  'Nguefack',
  'Tondji',
  5356,
  7394,
  3722,
  4545,
  3735,
  6790,
  6899,
  7314,
  8707],
 '1': ['6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d49c01e52ddb7875b4b',
  'd4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35',
  '4e07408562bedb8b60ce05c1decfe3ad16b72230967de01f640b7e4729b49fce',
  '4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328cb08b5531fcacdabf8a',
  'ef2d127de37b942baad06145e54b0c619a1f22327b2ebbcfbec78f5564afe39d',
  'e7f6c011776e8db7cd330b54174fd76f7d0216b612387a5ffcfb81e6f0919683',
  '7902699be42c8a8e46fbbb4501726517e86b22c56a189f7625a6da49081b2451',
  '2c624232cdd221771294dfbb310aca000a0df6ac8b66b696d90ef06fdefb64a3',
  '19581e27de7ced00ff1ce50b2047e7a567c76b1cbaebabe5ef03f7c3017bb5b7',
  '4a44dc15364204a80fe80e9039455cc1608281820fe2b24f1e5233ade6af1dd5',
  '4fc82b26aecb47d2868c4efbe3581732a3e7cbcc6c2efb32062c08170a05eeb8'

In [111]:
def Return_Elt(d,e):
    list_ = [] 
    A=d[str(int((log(len(d[str(0)]))/log(2))+1))]
    if e not in d[str(0)]:
        print("{} n appartient pas a la liste".format(e))
    else:
        index = d[str(0)].index(e)
        if index%2==0:
            list_.append(d[str(1)][index])
            list_.append(d[str(1)][index+1])
            s = d[str(1)][index]
            t = d[str(1)][index+1]
        else:
            list_.append(d[str(1)][index-1])
            list_.append(d[str(1)][index])
            s = d[str(1)][index-1]
            t = d[str(1)][index]
        for i in range(2,int((log(len(d[str(0)]))/log(2)))+1):
            hach = Add_and_hash(s,t)
            index = d[str(i)].index(hach)
            if index%2==0:  
                list_.append(d[str(i)][index+1])
                s = d[str(i)][index]
                t = d[str(i)][index+1]
            else:
                list_.append(d[str(i)][index-1])
                s = d[str(i)][index-1]
                t = d[str(i)][index]      
    return list_,A[0]        

In [112]:
La,A=Return_Elt(d,15)
e=La[0]
La

['e629fa6598d732768f7c726b4b621285f9c3b85303900aa912017db7617d8bdb',
 'b17ef6d19c7a5b1ee83b907c595526dcb1eb06db8227d650d5dda0a9f4ce8cd9',
 'b60b5791504acf1bb624635069f906e3a85230fa049dcade451328ab171c279f',
 'f536192675783d67b6127b9bfcaa48248bf8c462ec09d8424aaa552b6ff4a80d',
 'a3f49b0543fca4792957ce7a36a63b46c92a4aa5990dd9e543c9a24209bc574a',
 '6854db90b9bc00c2e5cf9ded54eb17dd7f37c23318882ff7ed7c0c3eff682d36']

In [113]:

def Check_Membership(L,e,A):
    if e not in L:
        print('{} n appartient pas a l arbre'.format(e))
    else:    
        a = L[0]
        for i in range(1,len(L)):
            b = L[i]
            a = Add_and_hash(a,b)
        if a == A:
            return  True
        else:
            return  False
Check_Membership(La,e,A)

True

In [114]:
## Check if the number is prime or not
def Is_prime(p):
    result=[]
    j=0
    for i in range(1,p+1):
        if p%i==0:
            j+=1
    if j==2:
        return True
    return False 

In [115]:
def verifier_challenge(p):
    if Is_prime(p):
        return random.randint(1,p-1)
    print("Veillez etrer un nombre premier!!!!")

In [116]:
### Begin which the proof
#def prover(g,h,p,d,x,r):
    ##convert x in integer
#    if not Is_prime(p):
#        print('{} n est pas premier!!!!'.format(p))
#    else:
#        x=x%p
#        C=(pow(g,x)*pow(h,r))%p
#        k=random.randint(1,p-1)
#        t=random.randint(1,p-1)
#        R=(pow(g,k)*pow(h,t))%p
#        c=2   #verifier_challenge(p)
#        z=(k-c*x)%p
#        y=(t-c*r)%p
#        pi,A=Return_Elt(d,x) ## d represent the Merkle tree
#        elt=pi[int(d[str(0)].index(x))]
    
#    return z,y,pi,A,C,c,R,elt

In [117]:
p=5
h=2
g=3
q=3
x=12
r=3

In [124]:
# This is the prover algorithm
def prover(q,g,h,x,r):
    if not Is_prime(q):
        print('{} n est pas premier!!!!'.format(q))
    else:    
        C=pow(g,x)*pow(h,r)
        k=random.randint(1,q-1)
        t=random.randint(1,q-1)    
        #c=random.randint(1,q-1)
        c=verifier_challenge(p)    ## this is the challenge
        R=int(pow(g,k)*pow(h,t))
        print(k,t,c)
        z=k-c*x
        y=t-c*r
        pi,A=Return_Elt(d,x) ## d represent the Merkle tree
        elt=sha256(str(x)).hexdigest()
    return C,c,z,y,R,pi,A,elt

In [125]:
C,c,z,y,R,pi,A,elt=prover(3,3,2,20,3)
C,c,z,y,R,pi,A,elt

(2, 1, 2)


(27894275208.0,
 2,
 -38,
 -5,
 18,
 ['9400f1b21cb527d7fa3d3eabba93557a18ebe7a2ca4e471cfe5e4c5b4ca7f767',
  'f5ca38f748a1d6eaf726b8a42fb575c3c71f1864a8143301782de13da2d9202b',
  'a34b2f2ed11ab8a89ae1a286e9f869b608de5c7f3fa9e2055978629798a833ae',
  '66e3049c5b9377e8a854443f8868e20fb6692877a8491e5a04d663b96a67b83d',
  'bc0372b4906ff01a03db83c095ac13a92f69f0053351d87b6f6c8777e4148d6c',
  '4de21ca37dbe102931deeecae9ef0f959893e061d20628e259dd61314ca874c2'],
 'f2ca2cad893e8de0a05b5f504dcca7b0e62ad8e75d3ab22ab5e0431efd8b3203',
 'f5ca38f748a1d6eaf726b8a42fb575c3c71f1864a8143301782de13da2d9202b')

In [186]:
# this is the verifier Algorithm
def verifier(C,c,z,y,R,pi,A,elt,g,h):
    r=int(pow(C,c)*pow(g,z)*pow(h,y))
    bol=Check_Membership(pi,elt,A)
    print(r,R)
    if (R==r and bol==True):
        
        return True
    return False

In [187]:
verifier(C,c,z,y,R,pi,A,elt,g,h)

(18, 18)


True

In [188]:
def __main__():
    
    p=51
    h=2
    g=3
    q=31
    x=20
    r=3
    Data= d
    C,c,z,y,R,pi,A,elt=prover(q,g,h,x,r)
    
    if verifier(C,c,z,y,R,pi,A,elt,g,h)==True:
        print("L element {} appartient a Data".format(elt))
    else:
        print("L element {} n'appartient a Data".format(elt))
__main__()

(14, 16, 3)
(313456656384, 313456656384)
L element f5ca38f748a1d6eaf726b8a42fb575c3c71f1864a8143301782de13da2d9202b appartient a Data


# End of Demo